# Real-time Image Stitching

In [1]:
!nvidia-smi 

Sun Jul  3 08:34:52 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.82.01    Driver Version: 470.82.01    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install --upgrade --no-cache gdown
# https://drive.google.com/file/d/1i6maaUCbQpLVFYpDXHDBI8RDXq3CPaAn/view?usp=sharing
!gdown --id 1i6maaUCbQpLVFYpDXHDBI8RDXq3CPaAn

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for gdown: filename=gdown-4.5.1-py3-none-any.whl size=14933 sha256=d14ebc57620f3f1bd0c46a990ac87ae0cf813da7c994a5edf59065f8a80fcaa4
  Stored in directory: /tmp/pip-ephem-wheel-cache-02lx0pff/wheels/3d/ec/b0/a96d1d126183f98570a785e6bf8789fca559853a9260e928e1
Successfully built gdown
/opt/conda/lib/python3.7/site-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1i6maaUCbQpLVFYpDXHDBI8RDXq3CPaAn
To: /kaggle/working/panorama.zip
100%|███████████████████████████████████████| 75.6M/75.6M [00:00<00:00, 314MB/s]


In [3]:
!unzip panorama.zip
!rm panorama.zip

Archive:  panorama.zip
   creating: panorama/
   creating: panorama/hr/
  inflating: panorama/hr/040.jpg     
  inflating: panorama/hr/046.jpg     
  inflating: panorama/hr/006.jpg     
  inflating: panorama/hr/001.jpg     
  inflating: panorama/hr/002.jpg     
  inflating: panorama/hr/005.jpg     
  inflating: panorama/hr/009.jpg     
  inflating: panorama/hr/007.jpg     
  inflating: panorama/hr/008.jpg     
  inflating: panorama/hr/010.jpg     
  inflating: panorama/hr/011.jpg     
  inflating: panorama/hr/012.jpg     
  inflating: panorama/hr/016.jpg     
  inflating: panorama/hr/019.jpg     
  inflating: panorama/hr/014.jpg     
  inflating: panorama/hr/015.jpg     
  inflating: panorama/hr/018.jpg     
  inflating: panorama/hr/017.jpg     
  inflating: panorama/hr/041.jpg     
  inflating: panorama/hr/030.jpg     
  inflating: panorama/hr/039.jpg     
  inflating: panorama/hr/049.jpg     
  inflating: panorama/hr/029.jpg     
  inflating: panorama/hr/032.jpg     
  inflating: pan

In [4]:
# Modules
import tensorflow as tf
from tqdm import tqdm
from tensorflow.keras.layers import Activation, Input, Lambda
from tensorflow.keras.layers import Dense, concatenate
from tensorflow.keras.layers import BatchNormalization, Conv2D, Conv2DTranspose, Flatten
from tensorflow.keras.layers import UpSampling2D, add, Concatenate
from tensorflow.keras.layers import LeakyReLU, PReLU

from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.optimizers import Adam


from numpy import array
from numpy.random import randint
from skimage import data, io, filters
import numpy as np
import cv2
import tensorflow.keras.backend as K
import tensorflow_addons as tfa
import matplotlib.pyplot as plt

import os
import sys

%matplotlib inline
plt.switch_backend('agg')


In [5]:
def gt_images(images):
    images_gt = images
    return array(images_gt)

def in_images(images_real):
    # images = []
    images_in = images_real
    return array(images_in)

def adj_normalize(input_data):

    return (input_data.astype(np.float32) - 127.5)/127.5 
    
def deadj_normalize(input_data):
    input_data = (input_data + 1) * 127.5
    return input_data.astype(np.uint8)

def feed_path(path):
    directories = []
    if os.path.isdir(path):
        directories.append(path)
    for elem in os.listdir(path):
        if os.path.isdir(os.path.join(path,elem)):
            directories = directories + feed_path(os.path.join(path,elem))
            directories.append(os.path.join(path,elem))
    return directories
    
def get_imgs(dirs, ext):
    files = []
    file_names = []
    count = 0
    for d in dirs:
        for f in sorted(os.listdir(d)): 
            if f.endswith(ext):
                if f:
                    image = cv2.imread(os.path.join(d,f))
                    image = cv2.resize(image, (512, 256), interpolation = cv2.INTER_CUBIC)
                    # print(f)
                    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                    if len(image.shape) > 2:
                        files.append(image)
                        file_names.append(os.path.join(d,f))
                else:
                    break
                count = count + 1
    return files 

def get_test_imgs(dirs, ext):
    files = []
    file_names = []
    count = 0
    for d in dirs:
        for f in sorted(os.listdir(d)): 
            if f.endswith(ext):
                if f:
                    image = cv2.imread(os.path.join(d,f))
#                     image = cv2.resize(image, (512, 256), interpolation = cv2.INTER_CUBIC)
                    # print(f)
                    print(image.shape)
                    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                    if len(image.shape) > 2:
                        files.append(image)
                        file_names.append(os.path.join(d,f))
                else:
                    break
                count = count + 1
    return files   

def feed_data(directory, ext):

    files = get_imgs(feed_path(directory), ext)
    return files
    

def conc(lin1, lin2):
    final_in = []
    for in1, in2 in zip(lin1, lin2):
        # print(lin)
        # print((in1.shape), (in2.shape))
        try:
#             img_bwa = cv2.bitwise_and(in1, in2)
#             ret,tgtesh = cv2.threshold(img_bwa,0,255,cv2.THRESH_BINARY)
            f_in  =  np.concatenate((in1, in2), axis=2)
#             print(f_in.shape)
            final_in.append(f_in)
        except Exception as e:
            print(f'Exception as {e}')
            pass
    return final_in

def load_training_data(gt_dir, in_dir1, in_dir2, ext, number_of_images = 46, train_test_ratio = 0.8):

    number_of_train_images = int(number_of_images * train_test_ratio)
    
    files = get_imgs(feed_path(gt_dir), ext)
    in1files = get_imgs(feed_path(in_dir1), ext)
    in2files = get_imgs(feed_path(in_dir2), ext)
    in_files = conc(in1files, in2files)
    
    if len(files) < number_of_images:
        print("No. of image files are less then you specified")
        print(f"Please reduce number of images to {len(files)}")
        sys.exit()
        
    test_array = array(files, dtype=object)
    # print(f'{len(test_array)}')
    if len(test_array) < 3:
        print("Images are of not same shape")
        print("Please provide same shape images")
        sys.exit()
    
    x_train = files[:number_of_train_images]
    x_test = files[number_of_train_images:number_of_images]
    x_in_train = in_files[:number_of_train_images]
    x_in_test = in_files[number_of_train_images:number_of_images]
    # print(x_in_test[0].shape) 
    x_train_gt = gt_images(x_train)
    x_train_gt = adj_normalize(x_train_gt)
  
    x_train_in = in_images(x_in_train)
    x_train_in = adj_normalize(x_train_in)
    # print(x_train_in.shape)
    
    x_test_gt = gt_images(x_test)
    x_test_gt = adj_normalize(x_test_gt)
    
    x_test_in = in_images(x_in_test)
    x_test_in = adj_normalize(x_test_in)
    # print(x_train_in.shape[0])
    
    return x_train_in, x_train_gt, x_test_in, x_test_gt


def load_test_data_for_model(gt_dir, in_dir1, in_dir2, ext, number_of_images = 5):

    files = get_test_imgs(feed_path(gt_dir), ext)
    in1files = get_test_imgs(feed_path(in_dir1), ext)
    in2files = get_test_imgs(feed_path(in_dir2), ext)
    in_files = conc(in1files, in2files)
    
    if len(files) < number_of_images:
        print("No. of image files are less then you specified")
        print(f"Please reduce number of images to {len(files)}")
        sys.exit()
        
        
    x_test_gt = gt_images(files)
    x_test_gt = adj_normalize(x_test_gt)
    
    x_test_in = in_images(in_files)
    x_test_in = adj_normalize(x_test_in)
    
    return x_test_in, x_test_gt
    
def load_test_data(in_dir1, in_dir2, ext, number_of_images = 100):

    files = get_imgs(feed_path(in_dir1), ext)
    files1 = get_imgs(feed_path(in_dir2), ext)
    in_files = conc(files, files1)

    if len(files) < number_of_images:
        print("No. of image files are less then you specified")
        print(f"Please reduce number of images to {len(files)}")
        sys.exit()
        
        
    x_test_in = in_images(in_files)
    x_test_in = adj_normalize(x_test_in)
    
    return x_test_in
    
def get_generated_images(output_dir, epoch, generator, x_test_gt, x_test_in , dim=(1, 3), figsize=(15, 5)):
    
    examples = x_test_gt.shape[0]
    value = randint(0, examples)
    image_batch_gt = deadj_normalize(x_test_gt)
    image_batch_in = x_test_in
    gen_img = generator.predict(image_batch_in)
    generated_image = deadj_normalize(gen_img)
    image_batch_in = deadj_normalize(image_batch_in)
    
    plt.figure(figsize=figsize)
    
    plt.subplot(dim[0], dim[1], 1)
    plt.imshow(image_batch_in[value], interpolation='nearest')
    plt.axis('off')
        
    plt.subplot(dim[0], dim[1], 2)
    plt.imshow(generated_image[value], interpolation='nearest')
    plt.axis('off')
    
    plt.subplot(dim[0], dim[1], 3)
    plt.imshow(image_batch_gt[value], interpolation='nearest')
    plt.axis('off')
    
    plt.tight_layout()
    plt.savefig(output_dir + 'generated_image_%d.png' % epoch)
    
    #plt.show()
    

def get_test_gen_images_for_model(output_dir, generator, x_test_gt, x_test_in , dim=(2, 1), figsize=(1000, 2000)):
    
    examples = x_test_gt.shape[0]
    image_batch_gt = deadj_normalize(x_test_gt)
    image_batch_in = x_test_in
    gen_img = generator.predict(image_batch_in)
    generated_image = deadj_normalize(gen_img)
    image_batch_in = deadj_normalize(image_batch_in)
    plt.figure(figsize=(15,10))
    for index in range(examples):
        cv2.imwrite(f'{output_dir}test_generated_image_{index}.png', cv2.cvtColor(generated_image[index], cv2.COLOR_BGR2RGB))


def plot_test_gen_images(output_dir, generator, x_test_in, figsize=(5, 5)):
    
    examples = x_test_in.shape[0]
    image_batch_in = deadj_normalize(x_test_in)
    gen_img = generator.predict(image_batch_in)
    generated_image = deadj_normalize(gen_img)
    
    for index in range(examples):
        cv2.imwrite(f'{output_dir}test_generated_image_{index}.png', cv2.cvtColor(generated_image[index], cv2.COLOR_BGR2RGB))

In [6]:
# Residual block
# @tf.function
def Residual_block(model, kernal_size, filters, strides):

    gen = model

    model = Conv2D(filters=filters, kernel_size=kernal_size,
                   strides=strides, padding="same")
#     model = BatchNormalization(momentum=0.5)(model)
    model = tfa.layers.WeightNormalization(model)(gen)
    # Using Parametric ReLU
    model = PReLU(alpha_initializer='zeros', alpha_regularizer=None,
                  alpha_constraint=None, shared_axes=[1, 2])(model)
    gen2 = model
    
    model = Conv2D(filters=filters, kernel_size=kernal_size,
                   strides=strides, padding="same")
    model = tfa.layers.WeightNormalization(model)(gen2)

    model = add([gen, model])

    return model

# @tf.function
def discriminator_based_block(model, filters, kernel_size, strides):

    first_model = model

    model = Conv2D(filters=filters, kernel_size=kernel_size,
                   strides=strides, padding="same")(model)
#     model = tfa.layers.WeightNormalization(model)(first_model)
    model = BatchNormalization(momentum=0.5)(model)
    model = LeakyReLU(alpha=0.2)(model)

    return model

In [7]:
@tf.function
def charbonnier_loss(y_true, y_pred):
    return tf.reduce_mean(tf.sqrt(tf.square(y_true - y_pred) + tf.square(1e-3)))

In [8]:
class Generator(object):

        
    # @tf.function
    def generator(self, noise_shape):

        gen_input = Input(shape=noise_shape)

        model = Conv2D(filters=32, kernel_size=8,
                       strides=2, padding="same")(gen_input)
        model = PReLU(alpha_initializer='zeros', alpha_regularizer=None,
                      alpha_constraint=None, shared_axes=[1, 2])(model)
        get_model = model
        model = Conv2D(filters=64, kernel_size=8,
                       strides=2, padding="same")(model)
        model = PReLU(alpha_initializer='zeros', alpha_regularizer=None,
                      alpha_constraint=None, shared_axes=[1, 2])(model)
        sec_model = model

        # Using 16 Residual Blocks
        for index in range(4):
            model = Residual_block(model, 3, 64, 1)
        
        res_model = model

        model = Conv2D(filters = 64, kernel_size = 3, strides = 1, padding = "same")
        model = tfa.layers.WeightNormalization(model)(res_model)
        
        model = add([sec_model, model])
        model = UpSampling2D(size=(2, 2), interpolation='bilinear')(model)
        model = Conv2D(filters=64, kernel_size=8,
                       strides=1, padding="same")(model)
        model = PReLU(alpha_initializer='zeros', alpha_regularizer=None,
                      alpha_constraint=None, shared_axes=[1, 2])(model)
        
#         model = Add()([get_model, model])
        model = Concatenate()([get_model, model])
        model = UpSampling2D(size=(2, 2), interpolation='bilinear')(model)
        model = Conv2D(filters=32, kernel_size=8,
                       strides=1, padding="same")(model)
        model = PReLU(alpha_initializer='zeros', alpha_regularizer=None,
                      alpha_constraint=None, shared_axes=[1, 2])(model)
#         model = Concatenate()([get_model, model])

        model = Conv2D(filters=3, kernel_size=3,
                       strides=1, padding="same")(model)
        model = Activation('tanh')(model)
        
        generator_model = Model(inputs=gen_input, outputs=model)

        return generator_model

In [9]:
class Discriminator(object):

    def discriminator(self, image_shape):

        dis_input = Input(shape=image_shape)

        model = Conv2D(filters=64, kernel_size=3,
                       strides=1, padding="same")(dis_input)
        model = LeakyReLU(alpha=0.2)(model)

        model = discriminator_based_block(model, 64, 3, 2)
        model = discriminator_based_block(model, 128, 3, 1)
        model = discriminator_based_block(model, 128, 3, 2)
        model = discriminator_based_block(model, 256, 3, 1)
        model = discriminator_based_block(model, 256, 3, 2)
        model = discriminator_based_block(model, 512, 3, 1)
        model = discriminator_based_block(model, 512, 3, 2)

        model = Flatten()(model)
        model = Dense(1024)(model)
        model = LeakyReLU(alpha=0.2)(model)

        model = Dense(1)(model)
        model = Activation('tanh')(model)

        discriminator_model = Model(inputs=dis_input, outputs=model)

        return discriminator_model

In [10]:
tf.config.run_functions_eagerly(True)
class PERCEPTUAL_VGG_LOSS(object):

    def __init__(self, image_shape):
        
        self.image_shape = image_shape

    # computes VGG loss or content loss
    # @tf.function
    def vgg_loss(self, y_true, y_pred):
    
        vgg19 = VGG19(include_top=False, weights='imagenet', input_shape=self.image_shape)
        vgg19.trainable = False
        # Make trainable as False
        for l in vgg19.layers:
            l.trainable = False
        model = Model(inputs=vgg19.input, outputs=vgg19.get_layer('block5_conv4').output)
        model.trainable = False
    
        return K.mean(K.square(model(y_true) - model(y_pred)))
    
def get_optimizer():
#         adam = Adam(learning_rate=1E-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
    adam = tfa.optimizers.AdamW(learning_rate=1E-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08, weight_decay=1E-5)
    return adam

In [11]:
np.random.seed(10)
# Remember to change image shape if you are having different size of images
image_shape = (None, None,6)

# Combined network
# @tf.function
def get_gan_network(discriminator, shape, generator, optimizer, loss):
    discriminator.trainable = False
    gan_input = Input(shape=shape)
    x = generator(gan_input)
    # print(x.shape)
    gan_output = discriminator(x)
    gan = Model(inputs=gan_input, outputs=[x,gan_output])
    gan.compile(loss=loss,
                loss_weights=[1., 1e-3],
                optimizer=optimizer)

    return gan
# @tf.function
def train(epochs, batch_size, gt_dir, in_dir1, in_dir2 , output_dir, model_save_dir, number_of_images, train_test_ratio):
    
    x_train_in, x_train_gt, x_test_in, x_test_gt = load_training_data(gt_dir, in_dir1, in_dir2 , '.jpg', number_of_images, train_test_ratio) 
    loss = PERCEPTUAL_VGG_LOSS((image_shape[0], image_shape[1], 3))  
    
    batch_count = int(x_train_gt.shape[0] / batch_size)
    # shape = (128, 64, 3)
    shape = (256, 512, 3)
    # print(x_train_in.shape[2])

    generator = Generator().generator(image_shape)
    discriminator = Discriminator().discriminator(shape)

    optimizer = get_optimizer()
    generator.compile(loss=[loss.vgg_loss, charbonnier_loss], optimizer=optimizer)
    discriminator.compile(loss=charbonnier_loss, optimizer=optimizer)
    
    gan = get_gan_network(discriminator, image_shape, generator, optimizer, charbonnier_loss)
    
    loss_file = open(model_save_dir + 'losses.txt' , 'w+')
    loss_file.close()

    for e in range(1, epochs+1):
        print ('-'*15, 'Epoch %d' % e, '-'*15)
        for _ in tqdm(range(batch_count)):
            
            random_nums = np.random.randint(0, x_train_gt.shape[0], size=batch_size)
            image_batch_gt = x_train_gt[random_nums]
            image_batch_in = x_train_in[random_nums]
            # print(x_train_gt[0].shape)
            generated_images_sr = generator.predict(image_batch_in)

            real_data_Y = np.ones(batch_size) - np.random.random_sample(batch_size)*0.2
            fake_data_Y = np.random.random_sample(batch_size)*0.2
            
            discriminator.trainable = True
            
            d_loss_real = discriminator.train_on_batch(image_batch_gt, real_data_Y)
            d_loss_fake = discriminator.train_on_batch(generated_images_sr, fake_data_Y)
            discriminator_loss = 0.5 * np.add(d_loss_fake, d_loss_real)
            
            random_nums = np.random.randint(0, x_train_gt.shape[0], size=batch_size)
            image_batch_gt = x_train_gt[random_nums]
            image_batch_in = x_train_in[random_nums]

            gan_Y = np.ones(batch_size) - np.random.random_sample(batch_size)*0.2
            discriminator.trainable = False
            gan_loss = gan.train_on_batch(image_batch_in, [image_batch_gt, gan_Y])
            
            
        print(f"discriminator_loss :  {discriminator_loss}")
        print(f"gan_loss : {gan_loss}")
        gan_loss = str(gan_loss)
        # discriminator.compile(loss=discriminator_loss, optimizer=optimizer)
        
        loss_file = open(model_save_dir + 'losses.txt' , 'a')
        loss_file.write(f'epoch {e} : gan_loss = {gan_loss} ; discriminator_loss = {discriminator_loss}\n')
        loss_file.close()

#         if e == 1 or e % 50 == 0:
#             get_generated_images(output_dir, e, generator, x_test_gt, x_test_in)
        if e % 500 == 0:
            generator.save(model_save_dir + 'get_model%d.h5' % e)
            discriminator.save(model_save_dir + 'dis_model%d.h5' % e)

In [12]:
epochs = 500
batch_size = 6
gt_dir = './panorama/hr/'
in_dir1 = './panorama/lr1/'
in_dir2 = './panorama/lr2/'
output_dir ='./'
model_save_dir = './'
number_of_images = 46
train_test_ratio = 0.9

In [13]:
train(epochs, batch_size, gt_dir, in_dir1, in_dir2, output_dir, model_save_dir, number_of_images, train_test_ratio)

2022-07-03 08:35:55.512928: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-03 08:35:55.616150: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-03 08:35:55.616865: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-03 08:35:55.618355: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

--------------- Epoch 1 ---------------


  0%|          | 0/6 [00:00<?, ?it/s]/opt/conda/lib/python3.7/site-packages/tensorflow/python/data/ops/dataset_ops.py:4212: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "
2022-07-03 08:35:59.434058: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2022-07-03 08:36:00.276816: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005
100%|██████████| 6/6 [00:15<00:00,  2.56s/it]


discriminator_loss :  0.49294571951031685
gan_loss : [0.3168630301952362, 0.316792368888855, 0.07067519426345825]
--------------- Epoch 2 ---------------


100%|██████████| 6/6 [00:05<00:00,  1.04it/s]


...

100%|██████████| 6/6 [00:05<00:00,  1.20it/s]


discriminator_loss :  0.4975813142955303
gan_loss : [0.07190247625112534, 0.07178634405136108, 0.11613357812166214]
--------------- Epoch 500 ---------------


100%|██████████| 6/6 [00:05<00:00,  1.08it/s]
/opt/conda/lib/python3.7/site-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)
2022-07-03 09:20:26.257082: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 1073741824 exceeds 10% of free system memory.


discriminator_loss :  0.4610885325819254
gan_loss : [0.06670945137739182, 0.0666067823767662, 0.10266970843076706]


2022-07-03 09:20:27.467132: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 1073741824 exceeds 10% of free system memory.
2022-07-03 09:20:31.763824: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 1073741824 exceeds 10% of free system memory.
2022-07-03 09:20:36.236594: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 1073741824 exceeds 10% of free system memory.
2022-07-03 09:20:38.360796: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 1073741824 exceeds 10% of free system memory.


In [14]:
!ls 

__notebook_source__.ipynb  get_model500.h5  panorama
dis_model500.h5		   losses.txt


In [15]:
out_dir = 'output'
os.mkdir(out_dir)

In [16]:
def test_model(gt_dir, in_dir1, in_dir2, model, number_of_images, output_dir):
    
    x_test_in, x_test_gt = load_test_data_for_model(gt_dir, in_dir1, in_dir2, 'jpg', number_of_images)
    get_test_gen_images_for_model(output_dir, model, x_test_gt, x_test_in)

def test_model_for_in_images(in_dir1, in_dir2 , model, number_of_images, output_dir):
    x_test_in = load_test_data(in_dir1, in_dir2, 'jpg', number_of_images)
    plot_test_gen_images(output_dir, model, x_test_in)

In [ ]:
gt_dir = './panorama/hr/'
in_dir1 = './panorama/lr1/'
in_dir2 = './panorama/lr2/'
output_dir = f'./{out_dir}/'
model_dir = './get_model500.h5'
number_of_images = 1

In [ ]:
loss = PERCEPTUAL_VGG_LOSS(image_shape)  
model = load_model(model_dir , custom_objects={'vgg_loss': loss.vgg_loss})
 
test_model(gt_dir, in_dir1, in_dir2, model, number_of_images, output_dir)
# loss = PERCEPTUAL_VGG_LOSS(image_shape)  
# model = load_model(model_dir , custom_objects={'vgg_loss': loss.vgg_loss})
 
# test_model_for_in_images(in_dir1, in_dir2, model, number_of_images, output_dir)

In [ ]:
!ls  test

In [67]:
!rm -r testhr
!rm -r test0
!rm -r test

In [68]:
import os
os.mkdir('test')

In [69]:
os.mkdir('test0')

In [70]:
os.mkdir('testhr')

In [71]:
!cp   ./panorama/lr2/043.jpg  ./test/043.jpg 
!cp   ./panorama/lr1/043.jpg  ./test0/043.jpg 
!cp   ./panorama/hr/043.jpg  ./testhr/043.jpg 

In [64]:
!ls

__notebook_source__.ipynb  losses.txt  test			   testhr
dis_model500.h5		   output      test0
get_model500.h5		   panorama    test_generated_image_0.png


In [ ]:
# !zip -r output.zip output

In [72]:
gt_dir = './testhr/'
in_dir1 = './test0/'
in_dir2 = './test/'

output_dir = './'
model_dir = './get_model500.h5'
number_of_images = 1

In [73]:
loss = PERCEPTUAL_VGG_LOSS((image_shape[0], image_shape[1], 3))  
model = load_model(model_dir , custom_objects={'vgg_loss': loss.vgg_loss, 'charbonnier_loss':charbonnier_loss})
 
test_model(gt_dir, in_dir1, in_dir2, model, number_of_images, output_dir)

(480, 1247, 3)
(480, 1247, 3)
(480, 1247, 3)


In [ ]:
!ls